In [ ]:
# API Requests
import pprint
import requests
import json
import time

# Data Science
import pandas as pd
import numpy as np
import scipy.stats as sc
# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Geocoding
from api_keys import opencage_key
from api_keys import geoapify_key


In [ ]:
# Create a file path
filepath_cards = "../Resources/cards_data.csv"
filepath_users = "../Resources/users_data.csv"
# Read in the data.
df_cards = pd.read_csv(filepath_cards)
df_users = pd.read_csv(filepath_users)

In [ ]:
df_cards.info()
df_users.info()

In [ ]:
# Merging noth files on ID column
df = pd.merge(df_cards, df_users, left_on="client_id", right_on="id", how="inner")
df.head()

In [ ]:
# Removing dollar sign from monetary columlns
df["credit_limit"] = df.credit_limit.str.strip("$")
df["per_capita_income"] = df.per_capita_income.str.strip("$")
df["yearly_income"] = df.yearly_income.str.strip("$")
df["total_debt"] = df.total_debt.str.strip("$")
df.head()

In [ ]:
# Converting objects to int value
df["credit_limit"] = df["credit_limit"].astype(int)
df["per_capita_income"] = df["per_capita_income"].astype(int)
df["yearly_income"] = df["yearly_income"].astype(int)
df["total_debt"] =df["total_debt"].astype(int)
df.info()

In [ ]:
# Remove duplicates by client_id
df2 = df.drop_duplicates(subset='client_id', keep='first')

# Checking the result after removing duplicates
df2.head()
df2.info()

In [ ]:
# Drop the specified columns
df2 = df2.drop(columns=['id_x', 'card_brand', 'card_type', 'card_number', 'expires', 'cvv', 'has_chip', 'num_cards_issued', 'credit_limit', 'acct_open_date', 'year_pin_last_changed', 'card_on_dark_web', 'id_y', 'retirement_age'])

# Checking the result after dropping the columns
df2.head()

In [ ]:
# Create new Debt to Income column

df2['debt_to_income'] = df2['total_debt'] / df2['yearly_income']
df2

In [ ]:
# Create the reverse Geocode function
def reverse_geocode(lat, lng, api_key):
    base_url = "https://api.opencagedata.com/geocode/v1/json"
    params = {
        "q": f"{lat},{lng}",
        "key": api_key,  # Using the API key passed from the imported file
        "language": "en",
        "pretty": 1
    }
    
    # Make the API request
    response = requests.get(base_url, params=params)
    
    # Debugging: Print out the status and the response body
    print(f"Requesting coordinates: {lat}, {lng}")
    print(f"Status Code: {response.status_code}")
    
    if response.status_code == 200:
        data = response.json()
        # Check if there are results
        if data['results']:
            print(f"Address found: {data['results'][0]['formatted']}")
            return data['results'][0]['formatted']
        else:
            print("No results found.")
            return "No results found"
    else:
        # Print the error message and response
        print(f"Error: {response.status_code}")
        print(response.text)  # This will print the response text for error details
        return f"Error: {response.status_code}"

# Add a 1-second delay to avoid overloading the API
def delayed_reverse_geocode(lat, lng, api_key):
    time.sleep(1)  # Sleep for 1 second between requests
    return reverse_geocode(lat, lng, api_key)

# Loop the reverse goecode through each row
df2['address'] = df2.apply(lambda row: delayed_reverse_geocode(row['latitude'], row['longitude'], opencage_key), axis=1)

# Check the result
df2.head()

In [ ]:
# Save the full DataFrame to a new CSV file
df2.to_csv('geocode_dataframe.csv', index=False)

In [ ]:
import re

# Create new dataframe
df3 = df2.copy()

# Define a pattern for state and zip code
address_pattern = r'(\w{2}) (\d{5})'

# Add a comma after the state abbreviation and before the zip code
def add_comma(address):
    # Search for the pattern of the state and zip code
    match = re.search(address_pattern, address)
    if match:
        # If a match is found, reformat the address with a comma between state and zip code
        state = match.group(1)
        zip_code = match.group(2)
        formatted_address = address.replace(f"{state} {zip_code}", f"{state}, {zip_code}")
        return formatted_address
    else:
        # Return the original address if no match is found
        return address

# Apply the function to the 'address' column to create the new formatted address
df3['address'] = df3['address'].apply(add_comma)

# Check the result
df3.head()

In [ ]:
# Define a function to extract the state abbreviation (two-letter state code)
def extract_state(address):
    # Regex pattern to capture the two-letter state abbreviation before the zip code
    match = re.search(r'\b([A-Z]{2})\b,\s*\d{5}', address)
    if match:
        return match.group(1)  # Return the state abbreviation
    else:
        return None  # Return None if no match is found

# Apply the function to the 'address' column to create the new 'state' column
df3['state'] = df3['address'].apply(extract_state)

# Reorder the columns to place 'state' directly to the right of 'address'
cols = ['client_id', 'current_age', 'birth_year', 'birth_month', 'gender', 'address', 'state', 'latitude', 'longitude', 
        'per_capita_income', 'yearly_income', 'total_debt', 'credit_score', 'num_credit_cards', 'debt_to_income']

df3 = df3[cols]

# Check the result
df3.head()

In [ ]:
# Identify the rows where the 'state' column has null values
null_state_rows = df3[df3['state'].isnull()]

# Display the rows with null values in the 'state' column
print(null_state_rows[['address', 'state']])

In [ ]:
# Remove rows with null values in the 'state' column
df3 = df3.dropna(subset=['state'])

# Check the result
df3.info()
df3

In [ ]:
# Save the full DataFrame to a new CSV file
df3.to_csv('full_dataframe.csv', index=False)